<a href="https://colab.research.google.com/github/bibowskii/LLM-T5-small-summarizer/blob/main/t5_trainer_round_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

preprocessing the text
cleaning
unused and unnecessary




In [4]:
from google.colab import drive
import time
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext




tokenizing using a pretrained model and loading our pretrained model


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')
modelT5 = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/LLMmodels/round3/epoch=9')

def tokenize(text):
    cleaned_text = text

    tokenized_text = tokenizer.encode(cleaned_text, return_tensors="pt")
    return tokenized_text

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


loading the dataset and splitting

In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
from datasets import load_dataset


trainingSet = load_dataset("ingoziegler/CRAFT-Summarization", split="train", name = 'XL' )


#checking the data
print(trainingSet[0])

README.md:   0%|          | 0.00/9.37k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/39.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

{'instruction': 'Summarize the given text into one or two sentences about the legal issues surrounding DMCA takedown notices, online reviews, and the music industry.', 'summary': "The text discusses various legal cases and issues related to DMCA takedown notices, online reviews, and the music industry. Ashley Madison attempted to use DMCA notices to prevent access to hacked data, but it's unclear if they can assert a copyright over factual information. Restaurants cannot prevent negative online reviews due to the First Amendment. The music industry's impact on Nashville was discussed, and the shift to digital distribution has affected songwriters.", 'long_but_clean_text': "The text includes discussions on several legal cases and issues surrounding DMCA takedown notices, online reviews, and the music industry. Ashley Madison, an extramarital affairs website, attempted to use DMCA notices to prevent access to hacked data, but it's unclear if they can assert a copyright over factual infor

function to tokenize the dataset using the preloaded tokenizer

In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelT5.to(device)
print(device)

cuda


In [10]:
def tokenize_dataset(dataset):
    inputs = tokenizer(
        dataset['long_but_clean_text'],
        return_tensors="pt",
        padding="max_length",  # Pad to max_length (only specified once)
        truncation=True,       # Truncate to max_length
        max_length=512        # Specify max_length for the article
    )
    targets = tokenizer(
        dataset['summary'],
        return_tensors="pt",
        padding="max_length",  # Pad to max_length (only specified once)
        truncation=True,       # Truncate to max_length
        max_length=200         # Specify max_length for the highlights
    )

    inputs['labels'] = targets['input_ids']
    inputs['decoder_attention_mask'] = targets['attention_mask']
    return inputs


applying tokenization and mapping the input to the output


In [11]:
tokenized_trainingSet = trainingSet.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

formatting the dataset

In [12]:
tokenized_trainingSet.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

verifying the datasets

In [13]:
example = tokenized_trainingSet[0]
print(example.keys())
print(tokenizer.decode(example['input_ids']))
print(tokenizer.decode(example['labels']))

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
The text includes discussions on several legal cases and issues surrounding DMCA takedown notices, online reviews, and the music industry. Ashley Madison, an extramarital affairs website, attempted to use DMCA notices to prevent access to hacked data, but it's unclear if they can assert a copyright over factual information. In another case, a Charleston, S.C. steakhouse asked diners to sign contracts holding them legally liable for negative reviews, but Clinical Fellow Andy Sellars pointed out that such contracts were meaningless. Regarding the music industry, Clinical Professor Christopher Bavitz spoke with Tom Vanderbilt for Outside Magazine about TripAdvisor and the role of Section 230 of the Communications Decency Act in immunizing platforms like TripAdvisor from certain kinds of liability for materials posted by the platform’s users. The Seaton case against TripAdvisor was also mentioned, in which the C

data loader to test in batches

In [14]:
from torch.utils.data import DataLoader
trainingLoader = DataLoader(tokenized_trainingSet, batch_size=4, shuffle= False, num_workers=2)


trainer


In [15]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss

def train_model(model, trainLoader, optimizer, device):
  model.train()
  total_loss = 0
  for batch in trainLoader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    optimizer.zero_grad()
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  return total_loss / len(trainLoader)

epochs

In [16]:
checkpoint_path = "/content/drive/MyDrive/LLMmodels/round4/epoch={epoch}"
num_epochs = 3
optimizer = AdamW(modelT5.parameters(), lr=5e-5)
for epoch in range(num_epochs):
  start_time = time.time()
  print(f"Epoch {epoch+1}/{num_epochs}")
  train_loss = train_model(modelT5, trainingLoader, optimizer, device)
  print(f"Train Loss: {train_loss}")
  modelT5.save_pretrained(checkpoint_path.format(epoch=epoch+10))
  end_time = time.time()
  excution_time = (end_time - start_time)/60
  print('time taken for this epoch:  {:.2f} minutes' .format(excution_time))

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Train Loss: 0.9589469275426865
time taken for this epoch:  22.61 minutes
Epoch 2/3
Train Loss: 0.8701376563358307
time taken for this epoch:  22.64 minutes
Epoch 3/3
Train Loss: 0.8384928302288055
time taken for this epoch:  22.59 minutes
